In [1]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.cluster
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import random
from ciu import determine_ciu
import six
import sys
import os
sys.modules['sklearn.externals.six'] = six
from skrules import SkopeRules
import openml
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [2]:
df_meta = pd.read_csv('metadata_merged.csv', index_col=0)
df_meta=df_meta.drop(columns= 'best_technique')

df_lime=pd.read_csv('records_lime.csv', index_col=0).drop(columns = 'lime_Dataset')
df_ciu=pd.read_csv('records_ciu.csv', index_col=0).drop(columns = 'ciu_Dataset')
df_anchor=pd.read_csv('records_anchor.csv', index_col=0).drop(columns = 'anchor_Dataset')

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df_lime = pd.read_csv('records_lime.csv', index_col=0).drop(columns='lime_Dataset')
df_ciu = pd.read_csv('records_ciu.csv', index_col=0).drop(columns='ciu_Dataset')
df_anchor = pd.read_csv('records_anchor.csv', index_col=0).drop(columns='anchor_Dataset')

common_columns = ['Fidelity', 'Identity', 'Separability', 'Speed']

for col in common_columns:
    max_value = max(df_lime['lime_' + col].max(), df_ciu['ciu_' + col].max(), df_anchor['anchor_' + col].max())
    df_lime['lime_' + col] /= max_value
    df_ciu['ciu_' + col] /= max_value
    df_anchor['anchor_' + col] /= max_value

merged_df = df_lime.merge(df_ciu, on='folder').merge(df_anchor, on='folder')

merged_df['lime_score'] = merged_df['lime_Fidelity'] + merged_df['lime_Identity'] - merged_df['lime_Separability'] - merged_df['lime_Speed']
merged_df['ciu_score'] = merged_df['ciu_Fidelity'] + merged_df['ciu_Identity'] - merged_df['ciu_Separability'] - merged_df['ciu_Speed']
merged_df['anchor_score'] = merged_df['anchor_Fidelity'] + merged_df['anchor_Identity'] - merged_df['anchor_Separability'] - merged_df['anchor_Speed']

merged_df['best_dataset'] = merged_df[['lime_score', 'ciu_score', 'anchor_score']].idxmax(axis=1)

selected_datasets = merged_df[['folder', 'best_dataset']]
selected_datasets=selected_datasets.merge(df_meta, on='folder')
selected_datasets = selected_datasets.drop(columns='folder')


In [14]:
merged_df

,lime_Fidelity,lime_Identity,lime_Separability,lime_Speed,folder,ciu_Fidelity,ciu_Identity,ciu_Separability,ciu_Speed,anchor_Fidelity,anchor_Identity,anchor_Separability,anchor_Speed,lime_score,ciu_score,anchor_score,best_dataset
0,0.000000,0.833333,0.0,0.173122,307,0.0025,0.000000,0.0,0.001732,0.911255,0.75,0.047368,0.182042,0.660211,0.000768,1.431845,anchor_score
1,0.030000,1.000000,0.0,0.277433,1067,0.0050,0.190476,0.0,0.002333,0.929519,0.90,0.005263,0.155903,0.752567,0.193143,1.668353,anchor_score
2,0.017500,0.000000,0.0,0.001308,50,0.0000,1.000000,0.0,0.001284,1.000000,0.00,1.000000,0.001198,0.016192,0.998716,-0.001198,ciu_score
3,0.012500,1.000000,0.0,0.260070,32,0.0000,0.062500,0.0,0.001952,0.987610,0.90,0.005263,0.265595,0.752430,0.060548,1.616753,anchor_score
4,0.005000,0.971429,0.0,0.361559,1466,0.0150,0.000000,0.0,0.004519,0.995456,0.65,0.005263,0.301514,0.614869,0.010481,1.338679,anchor_score
5,0.000000,1.000000,0.0,0.103609,1459,0.0075,0.142857,0.0,0.001354,0.847095,0.25,0.121053,0.084225,0.896391,0.149003,0.891818,lime_score
6,0.027500,1.000000,0.0,0.528127,1050,0.0000,0.135135,0.0,0.004554,0.988980,0.80,0.052632,0.104848,0.499373,0.130582,1.631500,anchor_score
7,0.022500,1.000000,0.0,0.322501,1068,0.0000,0.190476,0.0,0.002348,0.983920,0.45,0.300000,0.034293,0.699999,0.188129,1.099628,anchor_score
8,0.000000,1.000000,0.0,0.350158,1467,0.0000,0.250000,0.0,0.002468,0.986144,0.80,0.105263,0.053445,0.649842,0.247532,1.627435,anchor_score
9,0.000000,1.000000,0.0,0.096268,40496,0.0050,0.000000,0.0,0.001306,0.567299,0.40,0.247368,0.034081,0.903732,0.003694,0.685849,lime_score


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, make_scorer, accuracy_score

X = selected_datasets.drop(['best_dataset'], axis=1)
y = selected_datasets['best_dataset']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_classifier = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy')

grid_search.fit(X, y)

print("Best parameters found: ", grid_search.best_params_)

best_model = grid_search.best_estimator_

y_pred = best_model.predict(X_test)

print("Classification Report for the Best Model:")
print(classification_report(y_test, y_pred))

Best parameters found:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Classification Report for the Best Model:
              precision    recall  f1-score   support

anchor_score       1.00      1.00      1.00        12
  lime_score       1.00      1.00      1.00         4

    accuracy                           1.00        16
   macro avg       1.00      1.00      1.00        16
weighted avg       1.00      1.00      1.00        16



In [ ]:
def predict_best_dataset(input_data):

    input_df = pd.DataFrame([input_data])
    input_df = input_df[X.columns]

    prediction = best_model.predict(input_df)

    return prediction[0][:-6]

In [ ]:
input_data = df_meta.drop(columns='folder').iloc[10].to_dict()
input_data_formatted = {k: round(v, 2) for k, v in input_data.items()}
input_data_formatted


predicted_dataset = predict_best_dataset(input_data_formatted)
print("Predicted Best model for your cause, considering your dataset is", predicted_dataset.upper())

In [ ]:
input_data = {'attr_to_inst': 0.74,
 'cat_to_num': 0.1,
 'freq_class.mean': 0.5,
 'freq_class.sd': 0.44,
 'inst_to_attr': 1,
 'max.mean': 0.595,
 'max.sd': 1.150,
 'min.mean': -0.8,
 'min.sd': 1.1,
 'nr_cor_attr':10.1,
 'nr_norm': 4.0,
 'sd.mean': 0.40,
 'sd.sd': 0.811}

input_data_formatted = {k: round(v, 2) for k, v in input_data.items()}


predicted_dataset = predict_best_dataset(input_data_formatted)
print("Predicted Best model for your cause, considering your dataset is", predicted_dataset.upper())